In [1]:
import io
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import IPython.display as display
import ipywidgets as widgets

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
%matplotlib widget

In [3]:
file_repr = dict()

In [4]:
upload_text = widgets.Label(value="Esperando archivo de datos.")
upload_file = widgets.FileUpload(accept=".csv", multiple=False)
upload_output = widgets.Output()
upload_output.clear_output()
upload_container = widgets.VBox([upload_text, upload_file, upload_output])

def file_detected(value):
    if value:
        filename = list(upload_file.value.keys())[0]
        upload_text.value = f"Cargado el archivo {filename} ({upload_file.value[filename]['metadata']['size']/1024:0.2f} kB)"
        data_pd = pd.read_csv(io.BytesIO(upload_file.value[filename]['content']))
        with upload_output:
            print(f"Creado Pandas DataFrame para {filename}.")
        file_repr.update({"filename": filename, "df": data_pd})
        

upload_file.observe(file_detected, names="value")

In [5]:
vis_text = widgets.Label(value="Selección de acción de visualización")
vis_table_button = widgets.Button(description="Mostrar tabla")
vis_fig_button = widgets.Button(description="Mostrar gráfico")
vis_hbox = widgets.HBox([vis_table_button, vis_fig_button])
vis_html = widgets.HTML()
vis_out = widgets.Output()
vis_vbox = widgets.VBox([vis_text, vis_hbox, vis_html, vis_out])


def on_click_table(button):
    vis_out.clear_output()
    html_io = io.StringIO()
    file_repr["df"].to_html(buf=html_io, classes='table table-striped')
    vis_html.value = html_io.getvalue()

def on_click_fig(button):
    vis_html.value = ""
    with vis_out:
        file_repr["df"].plot(x="OrigenLongitud", y="OrigenLatitud", kind="scatter")

vis_table_button.on_click(on_click_table)
vis_fig_button.on_click(on_click_fig)

In [6]:
tabs_widget = widgets.Tab()
tabs_widget.children = [upload_container, vis_vbox]
tabs_widget.set_title(0, "Carga")
tabs_widget.set_title(1, "Visualización")

display.display(tabs_widget)